![StaSteelExample1.png](StaSteelExample1.png)
![StaSteelExample1.png](StaSteelExample1_metaldeck.png)
![StaSteelExample1.png](StaSteelExample1_metaldeck_load.png)
![StaSteelExample1.png](StaSteelExample1_stud.png)
![StaSteelExample1.png](StaSteelExample1_stud2.png)

In [1]:
import CompositeBeam as comp
import math

### Variables

In [2]:
# Birimler N,mm
# Yapım aşaması yükleri
DeadLoad   = 3.221 #kN/m^2
LiveLoad_c = 1 #kN/m^2

# Kompozit çalışma aşaması yükleri
SuperDeadLoad = 0.0 #kN/m^2
LiveLoad      = 2.0 #kN/m^2
w_girder      = 1.0 #kN/m kiriş ağırlığı

# Beton bilgisi
f_ck     = 20     #N/mm2
wc       = 2500   #kg/m3

# Çelik malzeme bilgisi
Fy       = 235        #N/mm2
Fu_beam  = 235*1.2    #N/mm2

# Çelik enkesit özellikleri
Abeam    = 2868.8     #mm^2
Hbeam    = 200        #mm
Hweb     = 378.8      #mm
b_flange = 100        #mm
t_flange = 8.5        #mm
t_web    = 5.6        #mm
Ix       = 1940*10**4 #mm^4
Ycon     = 150        #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

# Kiriş uzunluk ve aralık bilgisi
Laralık  = 1250  #mm
Lbeam    = 4840  #mm

# Stud çivisi bilgisi
D_stud     = 19      #mm
H_stud     = 100     #mm
b_studhead = 32      #mm
t_studhead = 10      #mm
Fy_stud    = 235     #N/mm2 çelik ankrajın minimum(karakteristik) akma dayanımı
Fu_stud    = 235*1.2 #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı
Nw         = 2       #Kesit başlığının genişliğinde atılan stud çivisi adeti
IsWebAlignmentWelded=False #Stud çivisi kiriş gövdesi izasında mı kaynaklanıyor.

# Metal sac bilgisi
hr       = 50     #mm
wr       = 100    #mm
t_sac    = 1.0    #mm
IsParallel=True   #Metal sac kompozit kirişe paralel mi atılıyor

### Design Code Requirements

In [3]:
comp.ConcAvailableStressCheck(f_ck=f_ck)
comp.SteelAvailableStressCheck(Fy=Fy)
comp.MetalDeckCheck(hr=hr,wr=wr)
comp.ConcreteCoverAboveTopOfHeadedStudAnchorsChecks(hr=hr,t_sac=t_sac, h_stud=H_stud, Ycon=Ycon)
comp.StudCheck(Ds=D_stud, tf=t_flange, Hs=H_stud, IsWebAlignmentWelded=False)

20 N/mm^2 ≤ 20 N/mm^2 < 70 N/mm^2 √ - TSSDC 12.2.3(a)
235 N/mm^2 ≤ 460 N/mm^2 √ - TSSDC 12.2.3(c)
hr = 50mm ≤ 75mm √ - TSSDC 12.4.2.3
wr = 100mm ≥ 50mm √ - TSSDC 12.4.2.3
h_stud-hr-t_sac = 49.0mm ≥ 38mm √ - TSSDC 12.4.2.3
Ycon - h_stud-t_sac = 49.0mm ≥ 13mm √ - TSSDC 12.4.2.3
Ycon - hr = 100mm ≥ 50mm √ - TSSDC 12.4.2.3
Ds = 19mm ≤ 2.5*tf = 21.25mm √ - TSSDC 12.8.1
Ds = 19mm ≤ 19mm √ - TSSDC 12.8.1
Hs = 100mm ≥ 4*Ds = 76mm √ - TSSDC 12.8.2


True

### Yapım aşaması deplasman kontrolü

In [4]:
# Yapım aşaması
w_girder = 1.0 #kN/m kiriş ağırlığı
W_G = comp.calc_w_cdl(DeadLoad=DeadLoad, GirdersSpaceLength=Laralık/10**3) + w_girder
W_Q = comp.calc_w_ll(LiveLoad=LiveLoad_c, GirdersSpaceLength=Laralık/10**3)
W_G,W_Q

(5.02625, 1.25)

In [5]:
# Yapım aşaması sabit yüklerinden(döşeme ve çelik kiriş) kaynaklı deplasman
delta_cdl = comp.SimpleCompositeBeamDeflection(w=W_G, Lbeam=Lbeam, I=Ix)
delta_cdl

9.26

In [6]:
print(f"{delta_cdl} < {(Lbeam/360)}")

9.26 < 13.444444444444445


In [7]:
# Ters sehim miktarı
delta_c = comp.Camber(delta_cdl=delta_cdl, Lbeam=Lbeam)

delta_cdl = 9.26mm < 19mm or Lbeam = 4840 < 7600mm olduğu için ters sehime gerek yoktur.


In [8]:
# Yapım aşaması hareketli yüklerinden kaynaklı deplasman
delta_yll = comp.SimpleCompositeBeamDeflection(w=W_Q, Lbeam=Lbeam, I=Ix)
delta_yll

2.3

In [9]:
# Yapım aşaması toplam düşey yerdeğiştirme, super dead düşey yerdeğiştirmesi yapım öncesi olmadığı için 0.0
delta_ytl = comp.calc_Delta_TL(delta_cdl=delta_cdl, delta_sdl=0., delta_ll=delta_yll,delta_c=delta_c)
delta_ytl

11.56

In [10]:
delta_ytl < Lbeam/240

True

### Kompozit kirişin karakteristik eğilme momenti dayanımının belirlenmesi

In [11]:
# Kiriş enkesit kontrolü
alfa = Hbeam/t_web
print(f"alfa = {round(alfa,2)} ≤ 3.76*(E/Fy)^0.5 = {round(3.76*(2*10**5/Fy)**0.5,2)} √")

alfa = 35.71 ≤ 3.76*(E/Fy)^0.5 = 109.69 √


In [12]:
# Effektif döşeme genişliği
b_eff = comp.EffectiveSlabWidth(L=Lbeam, Lu=Laralık)
b_eff

1210.0

In [13]:
# Stud çivisiz beton yüksekliği
tc = comp.Calc_tc(hr=hr,Ycon=Ycon,IsParallel=True)
tc

125.0

In [14]:
# Stud çivisiz beton alanı
Ac = b_eff * tc
Ac

151250.0

In [15]:
Cs = comp.YieldBeamCapacity(fy=Fy,As=Abeam)
Cs

674168.0

In [16]:
Cc = comp.CrushConcCapacity(fck=f_ck,Ac=Ac)
Cc

2571250.0

In [17]:
Mn_design = comp.CompositeBeamDesignFlexuralCapacity(C_conc=Cc, T_steel=Cs, Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac = Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
Mn_design/10**6

a = 32.77433155080214 mm
Y2 = 133.61283422459894 mm
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.
φMn = φ * (min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a)) 
= 0.9 * 674168.0 *  258.61 = 156913647.501


156.9136475

In [19]:
Cf        = comp.Calc_Cf(Vbeam=Cs, Vcon=Cc)
a         = comp.Calc_a(Cf =Cf, fck = f_ck, b_eff = b_eff)
Y2        = comp.Calc_Y2(Ycon =Ycon, a =a)
C_flange  = comp.Calc_Cflange(t_flange=t_flange, b_flange = b_flange, BeamFy=Fy)
Y1        = comp.Calc_Y1(T_steel=Cs, C_conc=Cc, C_flange=C_flange, t_flange=t_flange, t_web=t_web, BeamFy=Fy)
C_web     = comp.Calc_Cweb(Y1=Y1, t_flange=t_flange, t_web=t_web, BeamFy=Fy)
Mn_web    = comp.PTEInWebMn(T_steel=Cs, C_conc=Cc, Cflange=C_flange, Cweb=C_web, Hbeam=Hbeam, t_flange=t_flange, Y1=Y1, Y2=Y2)
Mn_flange = comp.PTEInFlangeMn(C_conc=Cc, Cflange=C_flange, Hbeam=Hbeam, BeamAs=Abeam, BeamFy=Fy, Y1=Y1, Y2=Y2)
Mn_slab   = comp.PTEInSlabMn(Cf=Cf, Hbeam=Hbeam, hr=hr, tc=tc, a=a)
Mn_design = comp.CompositeBeamDesignFlexuralCapacity2(C_conc=Cc, T_steel=Cs,Mn_web=Mn_web, Mn_slab=Mn_slab, Mn_flange=Mn_flange, Y1=Y1, t_flange=t_flange, fi_b=0.9)
comp.CompositeBeamFlexuralCapacityCheck(M_demand=29.37*10**6, Mn_design=Mn_design)
Ec = comp.ConcreteYoungModules(fck=f_ck, wc=2500)
A_sa = 3.14 * D_stud**2 /4
Rg = comp.GetRg(StudsNumberInFlangeWidth=2, wr=wr, hr=hr, IsPitchParaleltoBeam=True)
N_studfullinteraction = math.ceil(V_üs / Qn) * 2
N_studfullinteraction

a = 32.77 mm
Y2 = 133.615 mm
Y1=-864.0615501519757 < 0 ==> Y1 = 0
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.
φMn = φ * Mn_slab = 0.9 * 174.353kNm = 156.918kNm
Talep/Kapasite oranı = 0.19


True

### Arttırılmış düşey yükler altında eğilme momenti kapasite talebi

In [20]:
W_superdeadload = comp.calc_w_sdl(SuperDeadLoad=SuperDeadLoad,GirdersSpaceLength=Laralık/10**3)
W_superdeadload

0.0

In [21]:
W_G_composite = W_G + W_superdeadload
W_G_composite

5.02625

In [22]:
W_Q_composite = comp.calc_w_ll(LiveLoad=LiveLoad, GirdersSpaceLength=Laralık/10**3)
W_Q_composite

2.5

In [23]:
W_TL = comp.calc_w_tl(w_dl=W_G_composite, w_ll=W_Q_composite)
W_TL

10.031500000000001

In [24]:
# Talep eğilme momenti dayanımı
M_demand = W_TL * Lbeam**2 /8
M_demand/10**6

29.374238300000005

### Talep/Kapasite oranı

In [25]:
flexuralcapacityratio = round(M_demand / Mn_design,2)
flexuralcapacityratio

0.19

### Başlıklı çelik ankrajların sayısı ve yerleşimi

In [26]:
Ec = comp.ConcreteYoungModules(fck=f_ck, wc=2500)
Ec

24037.73075812274

In [27]:
A_sa = 3.14 * D_stud**2 /4
A_sa

283.385

In [28]:
Rg = comp.GetRg(StudsNumberInFlangeWidth=2, wr=wr, hr=hr, IsPitchParaleltoBeam=True)
Rg

1.0

In [29]:
Rp = comp.GetRp(Hs = H_stud, hr =hr, t_studhead = 5, IsPitchParaleltoBeam=True)
Rp

0.75

In [30]:
Qn = comp.OneStudShearCapacity(Asa=A_sa, fck=f_ck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=Fu_beam)
Qn

Qn = 0.5 * Asa * (fck*Ec)**0.5 = 98244.579
Qn = 59935.927 > Rg*Rp*Asa*Fu = 98244.579 ==> Qn = 98244.579


59935.927

In [31]:
Cs,Cc

(674168.0, 2571250.0)

In [32]:
# Tam etkileşimli olduğu için
C = min(Cs,Cc)
C

674168.0

In [33]:
V_üs = C
V_üs

674168.0

In [34]:
#  Başlık 2 şerli olarak kullanılması öngörüldüğü için 2 ile çarpılarak maksimum moment ile eğilme momentinin sıfır olduğu en yakın nokta (mesnet) arasında kullanılacak çelik ankraj sayısı

N_studfullinteraction = math.ceil(V_üs / Qn) * 2
N_studfullinteraction

24

### Kısmi etkileşimli kabul edilip kompozitlik derecesi %25 alınırsa

In [35]:
TotalQn = C*0.25
TotalQn

168542.0

In [36]:
N_studpartialinteraction = math.ceil(TotalQn / Qn)*2
N_studpartialinteraction

6

In [37]:
Mn_design2 = comp.CompositeBeamDesignFlexuralCapacity(C_conc=TotalQn,T_steel=Cs,Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac = Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
Mn_design2/10**6

a = 8.193582887700535 mm
Y2 = 145.90320855614974 mm
Kompozit kiriş kısmi etkileşimlidir. Kompozitlik oranı %25'in altına inmemelidir.
Basınç bloğu başlıkta kabul edilirse; Y1(veya tf') = 10.76 mm
PTE kiriş gövdesindedir.Çelikteki basınç derinliği(Y1 veya tf') yeniden hesaplanacak...
Cflange = b_flange * t_flange * Fy = 199750.0 N
Y1 = ((T_steel - C_conc - 2*Cflange) / (2*t_web*Fy)) + t_flange = 48.82142857142857mm
Cweb = t_web * Fy * (Y1 - t_flange) = 53063.0 N
φMn = φ * (C_conc*(Y1+Y2) + 2*Cflange*(Y1 - 0.5*t_flange) + 2*Cweb*(0.5*(Y1-t_flange)) + T_steel*(0.5*Hbeam-Y1)) = 0.9 * [32819279.791 + 17806285.714 + 2139575.964 + 34502955.143] = 78541286.951 


78.54128695

In [38]:
M_demand/Mn_design2

0.3739974151263892

In [39]:
# TotalQn = ShearStudCapacity(N=N, Qn=Qn)
# Itr = Calc_I_tr(n=n, b_eff=beff,hr=hr,hc=hc,hb=Hbeam,Ab=BeamAs,Ibeam=Ibeam)
# Cf = Calc_Cf(Vbeam=T_steel,Vcon=C_conc)
# Ieff = Calc_I_eff(Ibeam=Ibeam, TotalQn=TotalQn, Cf=Cf, Itr=Itr)